# CAES Dispatch Model 

Authors: [Simon Hilpert](simon.hilpert@uni-flensburg.de) and Clemens Wingenbach <br>
Institute: Europa Universität Flensburg (EUF) <br>
Date: 05.11.2019 <br>




## Mathmatical Description

Linear formulation for generic storage. The endogenous model variables are indicated by bold letters. Note that the index for the set of storages $s \in S$ is omitted. 

**Parmeter**

* Efficiency charge: $\eta_{in}$ 
* Efficiency discharge $\eta_{out}$
* Efficiency when not operating $\eta_{stand}$
* Lower bounds of variables $\underline{p}_{out}, \underline{p}_{in}, \underline{l}$
* Upper bounds of variables $\overline{p}_{out}, \overline{p}_{in}, \overline{l}$

**Variables**


Storage Discharge:<br>
$\underline{p}_{out}(t) \leq \textbf{p}_{out}(t) \leq \overline{p}_{out}(t) \qquad \forall t \in T$

Storage Charge:<br>
$\underline{p}_{in}(t) \leq \textbf{p}_{in}(t) \leq  \overline{p}_{in}(t) \qquad \forall t \in T$

Storage Level:<br>
$\underline{l}(t) \leq \textbf{l}(t) \leq \overline{l}(t) \qquad \forall t \in T$

**Constraints**

Storage balance:<br>
$\textbf{l}(t) =  \textbf{l}(t-1) \cdot \eta_{stand}  - 1 / \eta_{out}  \cdot \textbf{p}_{out}(t) + \eta_{in} \cdot \textbf{p}_{in}(t) \qquad \forall t \in T \setminus \{0\}\\
 \textbf{l}(0) = 0.5 \cdot \overline{l}(t)$
 
**Objective function** 

The objective function is maximising the profit of operation:

$$
max \sum_t \textbf{p}_{out}(t)  \cdot price(t) - \textbf{p}_{în}(t)  \cdot price(t)
$$

## Model Implementation

In [ ]:
using CSV
using Clp, JuMP
using Plots
using DataFrames

In [ ]:
df = CSV.read("data/caes-system/caes-system.csv", header=true, copycols=true);

In [ ]:
prices = CSV.read("data/shadow_prices.csv");

In [ ]:
plot(sort(prices[:DE_electricity] ,rev=true), color="black", linewidth=2.0, linestyle=:solid, 
    xlabel = "Hours of the year", ylabel = "Shadow Prices in Euro / MWh", label="Germany 2012 Spot")

Storage parameter for specific CAES Model:

 * $\eta_{in} = \eta_{out} = 0.7554$
 * $\overline{l} = 36.6\,\text{MWh}$
 * $\overline{p}_{out} = 120.75\,\text{MW}$
 * $\overline{p}_{in} = 241.5\,\text{MW}$
 
 Not used for LP representations, requires MILP:
 * $\underline{p}_{out} = 28.75\,\text{MW}$ 
 * $\underline{p}_{in} =  57.5\,\text{MW}$ 

In [ ]:
storages =  Dict(
                "storage1" => Dict(
                                "in" => Dict(
                                            "lb" => 0,
                                            "ub" => 120.75),
                                "out" => Dict(
                                            "lb" => 0,
                                            "ub" => 241.5),
                                "level" => Dict(
                                            "lb" => 0,
                                            "ub" => 36600), # MWh
                                "eta_stand" => 1,
                                "eta_in" => 0.755, 
                                "eta_out" => 0.755,
                                "init_level" => 0.5
    )
);

In [ ]:
m = Model(with_optimizer(Clp.Optimizer))

T = 8760;


In [ ]:
function storage_formulation(m, T, storages)

    p_out = @variable(m, 
                storages[s]["in"]["lb"] <= p_out[t=1:T, s=keys(storages)] <= storages[s]["in"]["ub"])
    
    p_in = @variable(m, 
                storages[s]["out"]["lb"] <= p_in[t=1:T, s=keys(storages)] <= storages[s]["out"]["ub"])

    level = @variable(m, storages[s]["level"]["lb"] <= level[t=1:T, s=keys(storages)] <= storages[s]["level"]["ub"])

    @constraint(m, [s=keys(storages)], level[1,s] == storages[s]["init_level"] * storages[s]["level"]["ub"])

    @constraint(m, [t=2:T,s=keys(storages)],
                level[t,s] == level[t-1,s] * storages[s]["eta_stand"] - 1/storages[s]["eta_out"] * p_out[t,s]  + storages[s]["eta_in"] * p_in[t,s])
    
    net_storage_sum = @expression(m, expr[t=1:T], 
                                  sum(p_in[t,s] for s in keys(storages)))
    
    return p_out, p_in, level, net_storage_sum
end
                    
p_out, p_in, level, net_storage_sum = storage_formulation(m, T, storages);

@objective(m, Max,
    sum(p_out[t, s] * prices[:DE_electricity][t] for t=1:T for s in keys(storages)) - 
    sum(p_in[t, s] * prices[:DE_electricity][t] for t=1:T for s in keys(storages))
);

In [ ]:
optimize!(m)

In [ ]:
results = DataFrame(
          timeindex=prices[:timeindex],
          prices=prices[:DE_electricity],
          level=JuMP.value.(m[:level][:,"storage1"].data),
          p_in=JuMP.value.(m[:p_in][:,"storage1"].data),
          p_out=JuMP.value.(m[:p_out][:,"storage1"].data))

CSV.write("results.csv",  delim=';', results);